### Imports

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
import random
import solara
from mesa.visualization import SolaraViz, make_space_component

### Agents

In [29]:
def get_pos_delta(curr_pos,next_pos):
    return (next_pos[0] - curr_pos[0], next_pos[1] - curr_pos[1])

def get_new_pos(curr_pos, delta):
    return (curr_pos[0] + delta[0], curr_pos[1] + delta[1])

class RobotAgent(Agent):
    
    def __init__(self,model):
        super().__init__(model)
        self.knowledge = lambda:0
        self.inventory = []
        self.ready_to_deliver = []
        self.inventory_full = False
        self.level = 0
    
    def get_level(self):
        return self.level
    
    def process_waste(self):
        self.ready_to_deliver.append(self.model.process_waste(self.inventory.pop(), self.inventory.pop()))

    def perceive(self):
        if len(self.inventory) > 1:
            self.inventory_full = True
        else:
            self.inventory_full = False
            
        possible_moves = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=True
        )

        self.knowledge.neighbors = [get_pos_delta(self.pos,possible_step) for possible_step in possible_moves]

        self.knowledge.close_contents = {}
        for possible_step in self.knowledge.neighbors:
            self.knowledge.close_contents[possible_step] = self.model.grid.get_cell_list_contents([get_new_pos(self.pos,possible_step)])

        self.knowledge.possible_moves = []
        for n in self.knowledge.neighbors:
            for a in self.knowledge.close_contents[n]:
                if isinstance(a,Radioactivity):
                    if a.get_radioactivity_level() == self.level:
                        self.knowledge.possible_moves.append(n)

    def deliberate(self):

        if self.inventory_full:
            self.process_waste()

        if len(self.ready_to_deliver):
            if (1,0) in self.knowledge.possible_moves:
                return (1,0)
            
            else:
                return "DROP"
        
        close_waste = {}
        for k in self.knowledge.possible_moves:
            waste_list = [w for w in self.knowledge.close_contents[k] if isinstance(w,Waste) if w.get_level()==self.get_level()]
            if len(waste_list) > 0:
                close_waste[k] = waste_list
        
        if (0,0) in close_waste.keys() and (1,0) in self.knowledge.possible_moves and not self.inventory_full:
            return "PICK"
        
        if len(close_waste.keys()) > 0:
            return random.choice(list(close_waste.keys()))
        
        return random.choice(self.knowledge.possible_moves)  

    def step(self):
        self.perceive()
        action = self.deliberate()
        self.model.perform_action(self,action)

class GreenRobotAgent(RobotAgent):
    def __init__(self, model):
        super().__init__(model)
        self.level = 1

class YellowRobotAgent(RobotAgent):
    def __init__(self, model):
        super().__init__(model)
        self.level = 2

class RedRobotAgent(RobotAgent):
    def __init__(self, model):
        super().__init__(model)
        self.level = 3

### Objects

In [26]:
class Radioactivity(Agent):
    def __init__(self,model, level = 1):
        super().__init__(model)
        self.level = level

    def get_radioactivity_level(self):
        return self.level
    
    def step(self):
        pass

class Waste(Agent):
    def __init__(self, model, level = 1):
        super().__init__(model)
        self.level = level
    
    def get_level(self):
        return self.level
    
    def step(self):
        pass

class DisposalZone(Agent):
    def __init__(self,model):
        super().__init__(model)
    
    def step(self):
        pass

### Model

In [ ]:
class RobotMission(Model):
    def __init__(self,n_agents=1, n_zone=3, n_waste=2, w=16, h=10, seed=None):
        super().__init__(seed=seed)
        
        self.n_agents = n_agents
        self.n_zone = n_zone
        self.n_waste = n_waste
        self.w = w # nombre de colones
        self.h = h # nombre de lignes

        # Mise en place de la MultiGrid
        self.grid = MultiGrid(self.w, self.h, torus = False)

        # Placement de la radioactivité
        Radioactivity.create_agents(model=self, n=(self.w//3*self.h), level=1)
        i = 0
        j = 0
        for agent in self.agents_by_type[type(Radioactivity(Model()))]:
            self.grid.place_agent(agent,(i%w, j%h))
            j += 1
            if j % h == 0:
                i += 1

        Radioactivity.create_agents(model=self, n=(self.w//3*self.h), level=2)
        for agent in self.agents_by_type[type(Radioactivity(Model()))]:
            if agent.get_radioactivity_level() == 2:    
                self.grid.place_agent(agent,(i%w, j%h))
                j += 1
                if j % h == 0:
                    i += 1

        Radioactivity.create_agents(model=self, n=(self.w//3*self.h), level=3)
        for agent in self.agents_by_type[type(Radioactivity(Model()))]:
            if agent.get_radioactivity_level() == 3:    
                self.grid.place_agent(agent,(i%w, j%h))
                j += 1
                if j % h == 0:
                    i += 1

        # Placement des Robots
        GreenRobotAgent.create_agents(self, n=self.n_agents)
        YellowRobotAgent.create_agents(self,n=self.n_agents)
        RedRobotAgent.create_agents(self,n=self.n_agents)

        for agent in self.agents:
            if isinstance(agent,RobotAgent):
                lvl = agent.get_level()
                w_r = random.randint((lvl-1)*(w//3),lvl*(w//3) - 1)
                h_r = random.randint(0,h-1)
                self.grid.place_agent(agent,(w_r,h_r))

        # Placement des déchets
        Waste.create_agents(self,n = self.n_waste, level = 1)
        Waste.create_agents(self,n = self.n_waste, level = 2)
        Waste.create_agents(self,n = self.n_waste, level = 3)

        for agent in self.agents_by_type[type(Waste(Model()))]:
            lvl = agent.get_level()
            w_w = random.randint((lvl-1)*(w//3),lvl*(w//3) - 1)
            h_w = random.randint(0,h-1)
            self.grid.place_agent(agent,(w_w,h_w))

        # Placement de la disposal zone
        DisposalZone.create_agents(self, n = self.h)
        h_dz = 0
        for agent in self.agents_by_type[type(DisposalZone(Model()))]:
            self.grid.place_agent(agent,(w-1,h_dz))
            h_dz += 1

    def perform_action(self, agent, action):
        if action == "PICK":
            waste = [w for w in self.grid.get_cell_list_contents([agent.pos]) if isinstance(w,Waste)][0]
            agent.inventory.append(waste)
            self.grid.remove_agent(waste)
        
        elif action == "DROP":
            waste = agent.ready_to_deliver.pop()
            self.grid.place_agent(waste,agent.pos)

        else:
            self.grid.move_agent(agent,get_new_pos(agent.pos,action))
    
    def process_waste(self,waste1,waste2):
        if waste1.get_level() == waste2.get_level():
            w = Waste.create_agents(self,1,level = (waste1.get_level()+1))
        return w[0]
    
    def step(self):
        self.agents.shuffle_do("step")

### Visualization

In [ ]:
def agent_portrayal(agent):

    if isinstance(agent,Radioactivity):
        if agent.get_radioactivity_level() == 1:
            return {
                "color": "#00FF0080",
                "size": 500,
                "marker": "s",
            }
        elif agent.get_radioactivity_level() == 2:
            return {
                "color": "#FFFF0080",
                "size": 500,
                "marker": "s",
            }
        elif agent.get_radioactivity_level() == 3:
            return {
                "color": "#FF000080",
                "size": 500,
                "marker": "s",
            }
    
    if isinstance(agent,RobotAgent):
        if agent.get_level() == 1:
            return {
                "color": "green",
                "size": 100,
            }
        if agent.get_level() == 2:
            return {
                "color": "orange",
                "size": 100,
            }
        if agent.get_level() == 3:
            return {
                "color": "red",
                "size": 100,
            }
    
    if isinstance(agent,Waste):
        if agent.get_level() == 1:
            return {
                "color": "green",
                "size": 50,
                "marker": "X"
            }
        
        if agent.get_level() == 2:
            return {
                "color": "orange",
                "size": 50,
                "marker": "X"
            }
        
        if agent.get_level() == 3:
            return {
                "color": "red",
                "size": 50,
                "marker": "X"
            }
        
        if agent.get_level() == 4:
            return {
                "color": "black",
                "size": 50,
                "marker": "X"
            }
    
    if isinstance(agent,DisposalZone):
        return {
            "color": "gray",
            "size": 500,
            "marker": "s"
        }
    
model_params = {
    "n_zone": 1,
    "n_agents": {
        "type": "SliderInt",
        "value": 1,
        "label": "Number of agents:",
        "min": 1,
        "max":5,
        "step":1
    },
    "n_waste": 1,
    "w": 10,
    "h": 10,
}

mod = RobotMission()

SpaceGraph = make_space_component(agent_portrayal)

page = SolaraViz(
    mod,
    components=[SpaceGraph],
    model_params=model_params
)

page

Cannot show ipywidgets in text

In [55]:
mod = RobotMission()
mod.step()

[(-1, -1), (0, 0), (1, 1)]
(-1, -1)
